In [1]:
from copy import deepcopy

import tqdm

In [2]:
input_file = "19_input.txt"

In [119]:
resource_names = ["ore", "clay", "obsidian", "geode"]

class Blueprint:
    def __init__(self, line):
        bp = line.rstrip().split()
        self.id = int(bp[1].rstrip(":"))
        costs = [int(bp[i]) for i in [6, 12, 18, 21, 27, 30]]
        self.costs = {
            # "wait": {"ore": 0, "clay": 0, "obsidian": 0},
            "ore": {"ore": costs[0], "clay": 0, "obsidian": 0, "geode": 0},
            "clay": {"ore": costs[1], "clay": 0, "obsidian": 0, "geode": 0},
            "obsidian": {"ore": costs[2], "clay": costs[3], "obsidian": 0, "geode": 0},
            "geode": {"ore": costs[4], "clay": 0, "obsidian": costs[5], "geode": 0}
        }
    
    def choices(self, state):
        options = []
        for robot in resource_names:
            if all(state.resources[res] >= cost for res, cost in self.costs[robot].items()):
                options.append(robot)
        return options + ["wait"]
                
    
    def find_best(self, nsteps=24):
        paths = [State(maxsteps=nsteps, costs=self.costs)]
        best = State()
        best.resources["geode"] = -1
        c = 0
        with tqdm.tqdm() as progbar:
            while paths:
                path = paths.pop()
                c += 1
                for build_robot in self.choices(path):
                    new_path = path.copy()
                    if build_robot == "wait":
                        new_path.wait()
                    else:
                        new_path.build(build_robot)
                    if new_path.time == nsteps:
                        if new_path.resources["geode"] > best.resources["geode"]:
                            best = new_path
                    else:
                        paths.append(new_path)
                if c % tq_update_rate == 0:
                    progbar.update(tq_update_rate)
                    progbar.set_description(f"Path n.{c}, time: {path.time}, queue: {len(paths)}")
        return best

tq_update_rate = 1000


class State:
    def __init__(self, maxsteps=24, costs=None):
        self.time = 0
        self.resources = {"ore": 0, "clay": 0, "obsidian": 0, "geode": 0}
        self.robots = {"ore": 1, "clay": 0, "obsidian": 0, "geode": 0}
        self.maxsteps = maxsteps
        self.costs = costs
    
    
    def build(self, robot):
        self.time += 1
        for res in self.resources:
            self.resources[res] += self.robots[res] - self.costs[robot][res]
        self.resources["geode"] += self.robots["geode"]
        self.robots[robot] += 1
    
    
    def wait(self):
        self.time += 1
        for res in self.resources:
            self.resources[res] += self.robots[res]
        

    def estimate_final(self):
        return (self.maxsteps - self.time) * self.robots["geode"] + self.resources["geode"]

    
    def copy(self):
        return deepcopy(self)

In [120]:
with open(input_file) as f:
    blueprints = [Blueprint(line) for line in f]

In [76]:
bestpath = blueprints[5].find_best(nsteps=15)

Path n.111000, time: 14, queue: 6: : 111000it [00:07, 13945.93it/s] 


In [77]:
print("Resources: ", bestpath.resources)
print("Robots: ", bestpath.robots)

Resources:  {'ore': 13, 'clay': 0, 'obsidian': 0, 'geode': 0}
Robots:  {'ore': 2, 'clay': 0, 'obsidian': 0, 'geode': 0}


In [78]:
bestpath.costs

{'ore': {'ore': 2, 'clay': 0, 'obsidian': 0, 'geode': 0},
 'clay': {'ore': 4, 'clay': 0, 'obsidian': 0, 'geode': 0},
 'obsidian': {'ore': 3, 'clay': 19, 'obsidian': 0, 'geode': 0},
 'geode': {'ore': 4, 'clay': 0, 'obsidian': 8, 'geode': 0}}

In [79]:
sample_bp1 = Blueprint("Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.")

In [121]:
bestsample = sample_bp1.find_best(nsteps=19)

Path n.3302000, time: 18, queue: 12: : 3302000it [04:39, 11796.19it/s]


In [118]:
print("Resources: ", bestsample.resources)
print("Robots: ", bestsample.robots)

Resources:  {'ore': 14, 'clay': 0, 'obsidian': 0, 'geode': 0}
Robots:  {'ore': 2, 'clay': 0, 'obsidian': 0, 'geode': 0}
